In [87]:
import pandas
import numpy as np
from scipy.stats import t as t_dist

In [88]:
split1 = np.loadtxt("./c1.txt")
split2 = np.loadtxt("./c2.txt")

Функция, которая принимает вектор сессий и возвращает вектор из 0 и 1 - были клики или нет

In [89]:
def have_clicks(split) :
    return (np.sum(split, axis=1) > 0).astype(int)

Функции, вычисляющая метрику CRT и Clicks @ 1. Делят split на группы по count сессий. И вычисляют  метрику в каждой группе. Размерность split кратна count.

In [129]:
def CTR(split, count) :
    sp_clicks = have_clicks(split)
    n = sp_clicks.shape[0]
    sp_clicks = sp_clicks.reshape(((n / count), count))
    return np.mean(sp_clicks, axis=1)
def clicks_on_1(split, count) :
    first_clicks = split[:, 0]
    n = first_clicks.shape[0]
    first_clicks = first_clicks.reshape(((n / count), count))
    return np.mean(first_clicks, axis=1)

In [139]:
print CTR(split1, 1000)[:10]
print CTR(split2, 1000)[:10]

[ 0.695  0.677  0.693  0.708  0.676  0.681  0.71   0.692  0.72   0.716]
[ 0.686  0.663  0.667  0.667  0.667  0.681  0.661  0.671  0.667  0.622]


In [141]:
print clicks_on_1(split1, 1000)[:10]
print clicks_on_1(split2,1000)[:10]

[ 0.479  0.472  0.501  0.503  0.461  0.481  0.511  0.502  0.511  0.507]
[ 0.328  0.326  0.327  0.321  0.314  0.346  0.311  0.337  0.359  0.312]


Критерий Стьюдента. Функция, которая возвращает 1, если гипотеза отвергается.

In [145]:
def students_t_test(X, Y, alpha=0.05) :
    """Возвращает 1, если гипотеза отвергается"""
    m = X.shape[0]
    n = Y.shape[0]
    X_mean = np.mean(X)
    Y_mean = np.mean(Y)
    sx_2 = np.var(X) * m / (m - 1)#np.var(X, ddof = 1)
    sy_2 = np.var(Y) * n / (n -1 )#np.var(Y, ddof = 1)
    s_2 = ((m - 1.0) * sx_2 + (n - 1.0) * sy_2) / (m + n - 2.0)
    t_value = ((X_mean - Y_mean) / (s_2 ** 0.5)) * ((m * n / (m + n)) ** 0.5)
    t = t_dist.ppf(q=1 - 0.05 / 2, df = m + n - 2)    
    return int(abs(t_value) > t) 

In [146]:
print students_t_test(CTR(split1, 1000), CTR(split2, 1000))

1


Бутстрепинг. Метрика подается параметром metric. Возвращает ASL.

In [151]:
def bootstrap(split1, split2, metric, count_iteration=7500, size_sample=50000, size_group=1000):
    i = 0
    sum_test_value = 0
    all_index1 = range(split1.shape[0])
    all_index2 = range(split2.shape[0])
    while i < count_iteration:
        index1 = np.random.choice(all_index1, size_sample, replace=True)
        index2 = np.random.choice(all_index2, size_sample, replace=True)
        sample1 = split1[index1]
        sample2 = split2[index2]
        
        x1 = metric(sample1, size_group)
        x2 = metric(sample2, size_group)
        sum_test_value += students_t_test(x1, x2)
        i += 1
    return float(sum_test_value) / count_iteration        

Проверка гипотезы H0 о том, что CTR на двух сплитах можно объяснить шумами.

In [152]:
bootstrap(split1, split2, CTR, count_iteration=1000, size_sample=50000, size_group=1000)

1.0

Проверка гипотезы H0 для метрики Clicks@1(доля кликов в первый результат).

In [153]:
bootstrap(split1, split2, clicks_on_1, count_iteration=1000, size_sample=50000, size_group=1000)

1.0

Обе гипотезы отвергнуты с уровнем значимости 1.